<a href="https://colab.research.google.com/github/Dibya069/Generative_AI-Codes-to-learn-/blob/main/RAG_%2B_PineCone_%2B_Memory_management_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain pinecone-client openai langchain-community

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2024-07-07-hallucination/")
data = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 100,
    length_function = len,
    add_start_index = True
  )

texts = text_splitter.split_documents(data)

In [ ]:
texts[:1]

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2024-07-07-hallucination/', 'title': "Extrinsic Hallucinations in LLMs | Lil'Log", 'description': 'Hallucination in large language models usually refers to the model generating unfaithful, fabricated, inconsistent, or nonsensical content. As a term, hallucination has been somewhat generalized to cases when the model makes mistakes. Here, I would like to narrow down the problem of hallucination to cases where the model output is fabricated and not grounded by either the provided context or world knowledge.\nThere are two types of hallucination:\nIn-context hallucination: The model output should be consistent with the source content in context.', 'language': 'en', 'start_index': 6}, page_content="Extrinsic Hallucinations in LLMs | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY']) # set openai_api_key = 'your_openai_api_key'

In [ ]:
from pinecone import Pinecone
import time
from pinecone import ServerlessSpec

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))


index_name = "hallusination"
index = pc.Index(index_name)

time.sleep(1)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-4', temperature=0)
llm.predict("what is LoRA")

<ipython-input-26-c3f3bd1f605c>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name='gpt-4', temperature=0)
<ipython-input-26-c3f3bd1f605c>:3: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("what is LoRA")


"LoRa (Long Range) is a wireless communication technology designed for long-range, low-power communications. It's primarily used in Internet of Things (IoT) networks, smart homes, and smart cities. It uses a spread spectrum modulation in the Sub-GHz band to enable long range (greater than 10 miles) coverage, low power consumption (up to 10 years battery life), and high network capacity (up to millions of messages per base station)."

In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.0 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import Pinecone

vectordb = Pinecone.from_documents(texts, embeddings, index_name=index_name)

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(memory_key="chat_history", return_messages= True)
chain = ConversationalRetrievalChain.from_llm(llm, retriever= retriever, memory= memory)

<ipython-input-54-628112435da8>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages= True)


In [ ]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "q"]:
        print("Goodbye!")
        break

    result = chain({"question": user_input})
    print("Bot:", result["answer"])
    print("\n")

User: what is Hallucination ?


<ipython-input-55-2f89df8375d0>:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain({"question": user_input})


Bot: Hallucination in large language models usually refers to the model generating unfaithful, fabricated, inconsistent, or nonsensical content. It has been somewhat generalized to cases when the model makes mistakes. Specifically, the term hallucination is used to describe cases where the model output is fabricated and not grounded by either the provided context or world knowledge.


User: Why it happen ??
Bot: Hallucination in large language models can occur due to issues at both the pre-training and fine-tuning stages. During pre-training, the model is exposed to a vast amount of data, often crawled from the public internet. This data can contain out-of-date, missing, or incorrect information, which the model may incorrectly memorize while maximizing the log-likelihood, leading to mistakes or hallucinations. In the context of fine-tuning, the model may generate unfaithful or fabricated content that is not grounded by the provided context or world knowledge, leading to what is referr

***

In [ ]:
!pip install langchain pinecone-client openai langchain-community tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/4

In [ ]:
import os
import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone as lang_pine
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from pinecone import Pinecone, ServerlessSpec
from transformers import AutoTokenizer, AutoModel

In [ ]:
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [ ]:
conversation_history = [
    "1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
    "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
    "3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
    "4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
    "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
    "6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
    "7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.",
    "8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.",
    "9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.",
    "10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.",
    "11: User: Hiking sounds fun! Any specific trails? | Bot: There are great trails in the Rockies and the Appalachian Mountains.",
    "12: User: I'll plan a trip. What about indoor activities? | Bot: Indoor activities like reading, cooking, or playing board games.",
    "13: User: Nice! Any good board games? | Bot: Settlers of Catan and Ticket to Ride are both excellent choices.",
    "14: User: I'll try them out. Any movie recommendations? | Bot: 'Inception' and 'The Matrix' are must-watch movies.",
    "15: User: I love those movies! Any TV shows? | Bot: 'Breaking Bad' and 'Stranger Things' are very popular.",
    "16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.",
    "17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.",
    "18: User: I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.",
    "19: User: That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind.",
    "20: User: Thanks for the tips! Any last advice? | Bot: Just enjoy your journey and make the most out of your experiences."
  ]

In [ ]:
def add_embeddings_to_pinecone(history, index_name='chat-history'):
    # Load openai model embeddings and pinecone
    embeddings = OpenAIEmbeddings(openai_api_key=os.environ.get("OPENAI_API_KEY"))
    pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

    # Create Pinecone index if it doesn't exist
    if index_name not in pc.list_indexes():
        pc.create_index(index_name, dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region='us-east-1'))


    # Encode and upsert messages into Pinecone index
    index = pc.Index(index_name)
    for i, message in enumerate(history):
        embedding = embeddings.embed_query(message)
        message_id = f"msg-{i}"
        index.upsert([(message_id, embedding)])

    print(index.describe_index_stats())

add_embeddings_to_pinecone(conversation_history)

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 19}},
 'total_vector_count': 19}


In [ ]:
def retrieve_relevant_history(query, history, index_name='chat-history'):
    # Load OpenAI model embeddings
    embeddings = OpenAIEmbeddings(openai_api_key=os.environ.get("OPENAI_API_KEY"))

    # Encode the query to get its embedding
    query_embedding = embeddings.embed_query(query)

    # Query Pinecone for relevant history
    pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
    index = pc.Index(index_name)

    # Retrieve top 3 relevant messages
    result = index.query(vector = query_embedding, top_k = 3, include_metadata = True)

    print(result)
    # Extract relevant message IDs and their associated metadata
    relevant_messages = []
    for match in result['matches']:
        message_id = match['id']
        # Fetch the relevant message by its ID
        relevant_messages.append(conversation_history[int(message_id.split('-')[1])])  # Ensure IDs are integers

    return relevant_messages


In [ ]:
def prepare_prompt(final_test_prompt, history):
  # Retrieve relevant history (message 7, 8, and 10) using Pinecone
  relevant_messages = retrieve_relevant_history(final_test_prompt, history)

  # Combine relevant history and final test prompt
  context_referred = "\n".join(relevant_messages)
  prepared_prompt = f"{context_referred}\nUser: {final_test_prompt}\nBot:"

  return prepared_prompt, context_referred

def generate_openai_response(prepared_prompt):
  llm = ChatOpenAI(model_name='gpt-4', temperature=0)
  response = llm.predict(prepared_prompt)

  return response

In [ ]:
# Call prepare_prompt
final_test_prompt = "Do you think it will help me stay fit?"
prepared_prompt, context_referred = prepare_prompt(final_test_prompt, conversation_history)


# Generate the response
response = generate_openai_response(prepared_prompt)

{'matches': [{'id': 'msg-6', 'score': 0.791899502, 'values': []},
             {'id': 'msg-7', 'score': 0.762535572, 'values': []},
             {'id': 'msg-8', 'score': 0.762237608, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}


In [ ]:
# Print the results
print(f"Final Test Prompt: {final_test_prompt} \n")
print(f"Context Referred: \n{context_referred} \n")
print(f"Final Test Prompt Response: {response} \n")

Final Test Prompt: Do you think it will help me stay fit? 

Context Referred: 
7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.
8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.
9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book. 

Final Test Prompt Response: Yes, 'Atomic Habits' provides valuable insights on how to build good habits and break bad ones, which can be very helpful in maintaining a regular exercise routine and staying fit. 



In [ ]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "q"]:
        print("Goodbye!")
        break

    prepared_prompt, context_referred = prepare_prompt(user_input, conversation_history)
    response = generate_openai_response(prepared_prompt)

    print(f"Context Referred: \n{context_referred} \n")
    print(f"Response: {response} \n")
    print("\n ------------------------------------------ \n")

User: Do you think it will help me stay fit? 
{'matches': [{'id': 'msg-6', 'score': 0.803957105, 'values': []},
             {'id': 'msg-7', 'score': 0.775736928, 'values': []},
             {'id': 'msg-8', 'score': 0.774762511, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}
Context Referred: 
7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.
8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.
9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book. 

Response: Yes, 'Atomic Habits' provides valuable insights into forming good habits, which can definitely help in maintaining a regular exercise routine and staying fit. 


 ------------------------------------------ 

User: why did you give me this type of movie recommendations ??
{'matches': [{'id': 'msg-13',

***

In [ ]:
def add_new_conversation_embedding_to_pinecone(message, index_name='chat-history'):
    # Load OpenAI model embeddings
    embeddings = OpenAIEmbeddings(openai_api_key=os.environ.get("OPENAI_API_KEY"))

    # Create Pinecone client and index
    pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
    index = pc.Index(index_name)

    # Get the embedding for the message
    embedding = embeddings.embed_query(message)

    # Generate a unique message ID based on the history length
    message_id = f"msg-{len(conversation_history)}"

    # Upsert the message embedding into Pinecone
    index.upsert([(message_id, embedding)])

    print(f"Added to Pinecone: {message_id}, {message}")

In [ ]:
def generate_openai_response(prepared_prompt, user_query):
    llm = ChatOpenAI(model_name='gpt-4', temperature=0)
    response = llm.predict(prepared_prompt)  # Pass the prepared prompt

    # Append user query and AI response to history
    conversation_history.append(f"user: {user_query} | bot: {response}")  # Append user query

    # Add both the user query and the generated response to Pinecone
    add_new_conversation_embedding_to_pinecone(f"user: {user_query} | bot: {response}")

    return response

In [ ]:
# Example of calling the functions
final_test_prompt = "Do you think it will help me stay fit?"

# Prepare the prompt
prepared_prompt, context_referred = prepare_prompt(final_test_prompt, conversation_history)

# Generate the response and update the conversation history
response = generate_openai_response(prepared_prompt, final_test_prompt)


{'matches': [{'id': 'msg-6', 'score': 0.791899502, 'values': []},
             {'id': 'msg-7', 'score': 0.762535572, 'values': []},
             {'id': 'msg-8', 'score': 0.762237608, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}
Added to Pinecone: msg-21, user: Do you think it will help me stay fit? | bot: Yes, 'Atomic Habits' provides valuable insights on how to build good habits and break bad ones, which can be very helpful in maintaining a regular exercise routine and staying fit.


In [ ]:
conversation_history

["1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
 "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
 '3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?',
 '4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.',
 "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
 '6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?',
 '7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.',
 '8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.',
 "9: User: I'll start with ru

***
***

In [ ]:
!pip install langchain pinecone-client openai langchain-community tiktoken sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import os
import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone as lang_pine
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from pinecone import Pinecone, ServerlessSpec
from transformers import AutoTokenizer, AutoModel

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [ ]:
conversation_history = [
    "1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
    "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
    "3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
    "4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
    "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
    "6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
    "7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.",
    "8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.",
    "9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.",
    "10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.",
    "11: User: Hiking sounds fun! Any specific trails? | Bot: There are great trails in the Rockies and the Appalachian Mountains.",
    "12: User: I'll plan a trip. What about indoor activities? | Bot: Indoor activities like reading, cooking, or playing board games.",
    "13: User: Nice! Any good board games? | Bot: Settlers of Catan and Ticket to Ride are both excellent choices.",
    "14: User: I'll try them out. Any movie recommendations? | Bot: 'Inception' and 'The Matrix' are must-watch movies.",
    "15: User: I love those movies! Any TV shows? | Bot: 'Breaking Bad' and 'Stranger Things' are very popular.",
    "16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.",
    "17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.",
    "18: User: I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.",
    "19: User: That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind.",
    "20: User: Thanks for the tips! Any last advice? | Bot: Just enjoy your journey and make the most out of your experiences."
  ]

In [ ]:
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
index = pc.Index("chat-history")
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 21}},
 'total_vector_count': 21}


In [ ]:
# Function to upscale the embedding to 1536 dimensions
def upscale_embedding(embedding, target_dim=1536):
    if len(embedding) < target_dim:
        repeat_times = target_dim // len(embedding)  # Number of times to repeat
        upscale_embedding = np.tile(embedding, repeat_times)
        return upscale_embedding[:target_dim]  # Ensure it's exactly the target dimension
    return embedding

# Function to retrieve relevant history using transformer embeddings
def retrieve_relevant_history(query, history, index_name='chat-history'):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')  # Embeddings are of size 384
    query_embedding = model.encode(query)

    query_embedding_1536 = upscale_embedding(query_embedding, target_dim=1536)

    pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
    index = pc.Index(index_name)

    print(query_embedding_1536)
    print(query_embedding_1536.tolist())

    result = index.query(vector=query_embedding_1536.tolist(), top_k=3, include_metadata=True)

    print(result)

    relevant_messages = []
    for match in result['matches']:
        message_id = match['id']
        relevant_messages.append(history[int(message_id.split('-')[1])])  # Ensure IDs are integers

    return relevant_messages

In [ ]:
def prepare_prompt(final_test_prompt, history):
  # Retrieve relevant history (message 7, 8, and 10) using Pinecone
  relevant_messages = retrieve_relevant_history(final_test_prompt, history)

  # Combine relevant history and final test prompt
  context_referred = "\n".join(relevant_messages)
  prepared_prompt = f"{context_referred}\nUser: {final_test_prompt}\nBot:"

  return prepared_prompt, context_referred

def generate_openai_response(prepared_prompt):
  llm = ChatOpenAI(model_name='gpt-4', temperature=0)
  response = llm.predict(prepared_prompt)

  return response

In [ ]:
# Call prepare_prompt
final_test_prompt = "Do you think it will help me stay fit?"
prepared_prompt, context_referred = prepare_prompt(final_test_prompt, conversation_history)


# Generate the response
response = generate_openai_response(prepared_prompt)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[-0.02380017 -0.05101923  0.06270394 ...  0.05499432 -0.00539855
 -0.01926768]
[-0.023800166323781013, -0.051019228994846344, 0.062703937292099, 0.056349385529756546, 0.0706346333026886, 0.10531844943761826, 0.02411370910704136, 0.007022933103144169, -0.1005902886390686, -0.004349107854068279, 0.0042559984140098095, 0.0204425398260355, 0.029890641570091248, -0.036711324006319046, 0.06387130916118622, -0.06271547824144363, 0.06150344759225845, 0.07029162347316742, -0.060439664870500565, 0.006917393300682306, -0.05456451699137688, 0.009178593754768372, 0.042486522346735, -0.0007094842148944736, -0.11575763672590256, 0.09969406574964523, -0.05169711261987686, -0.030750324949622154, -0.0659516230225563, -0.022947117686271667, -0.04976233094930649, 0.04048355296254158, 0.008567937649786472, 0.0034958533942699432, -0.011107470840215683, 0.03464455157518387, -0.00115306640509516, 0.0009511320968158543, -0.11449052393436432, 0.04772428423166275, -0.08972293883562088, 0.051088884472846985, 0.01

<ipython-input-8-7e1222a81051>:12: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name='gpt-4', temperature=0)
<ipython-input-8-7e1222a81051>:13: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(prepared_prompt)


In [ ]:
# Print the results
print(f"Final Test Prompt: {final_test_prompt} \n")
print(f"Context Referred: \n{context_referred} \n")
print(f"Final Test Prompt Response: {response} \n")

Final Test Prompt: Do you think it will help me stay fit? 

Context Referred: 
16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.
17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.
10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument. 

Final Test Prompt Response: Absolutely, especially hiking. It's a great way to stay active and enjoy nature. Learning a new instrument can also help improve mental fitness. 

